[![StackExchnage Codes](https://cdn.sstatic.net/Sites/math/Img/logo.svg?v=4a36350d1199)](https://github.com/RoyiAvital/StackExchangeCodes)

# StackExchange Mathematics  

## Q2154648 - Manual Implementation of the Kernel SVM to Match SciKit Learn

> Notebook by:
> - Royi Avital RoyiAvital@yahoo.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 16/08/2025 | Royi Avital | First version                                                      |

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.svm import SVC

# Optimization
import cvxpy as cp

# Miscellaneous
import math
from platform import python_version
import random

# Typing
from typing import Callable, List, Optional, Tuple, Union

# Visualization
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

In [ ]:
# Configuration

# warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

In [ ]:
# Auxiliary Functions

def Plot2DClsData( mX: np.ndarray, vY: np.ndarray, /, *, hA: Optional[plt.Axes] = None, figSize: Tuple[float, float] = (8.0, 6.0), colorMap: str = 'viridis' ) -> plt.Axes:
    """
    Plots 2D classification data.
    
    Parameters:
    - mX: 2D numpy array of shape (numSamples, 2) containing feature data.
    - vY: 1D numpy array of shape (numSamples,) containing class labels.
    """

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)

    colorMap = plt.get_cmap(colorMap)

    vQ = np.unique(vY)
    numCls = np.size(vQ)

    for valQ, vClsClr in zip(vQ, colorMap(np.linspace(0, 1, numCls))):
        vI = np.flatnonzero(vY == valQ)
        hA.scatter(mX[vI, 0], mX[vI, 1], s = 50, edgecolor = 'k', color = vClsClr, label = f'Class: {valQ}')
    
    hA.set_xlabel(r'$x_1$')
    hA.set_ylabel(r'$x_2$')
    
    return hA

def KernelSVMCVXPY( mK: np.ndarray, vY: np.ndarray, /, *, paramC: float = 1.0 ) -> Tuple[np.ndarray, float]:
    """
    Solves the dual problem of a Support Vector Machine using CVXPY.
    
    Parameters:
    - mK: Kernel matrix of shape (numSamples, numSamples).
    - vY: Class labels of shape (numSamples,).
    - paramC: Regularization parameter.
    
    Returns:
    - vβ: Optimization variables.
    - b: Bias term.
    """
    
    numSamples = mK.shape[0]
    
    # Define the dual variable
    vβ     = cp.Variable(numSamples)
    paramB = cp.Variable(1)
    # Define the objective function
    hingeLoss = cp.sum(cp.pos(1 - cp.multiply(vY, mK @ vβ + paramB)))
    # hingeLoss = cp.sum([cp.pos(1 - vY[ii] * (mK[ii] @ vβ + paramB)) for ii in range(numSamples)])
    cpObjFun  = cp.Minimize(0.5 * cp.quad_form(vβ, mK, assume_PSD = True) + (paramC * hingeLoss))
    # Define Constraints
    cpConst   = []
    # Define the Problem
    oCvxPrb   = cp.Problem(cpObjFun, cpConst)
    
    # Solve the problem
    # oCvxPrb.solve(solver = cp.SCS, verbose = False)
    oCvxPrb.solve(solver = cp.CLARABEL, verbose = False)
    
    return vβ.value, paramB.value[0].item()

In [ ]:
# Parameters

tuGridX = (-3, 3)
tuGridY = (-3, 3)
numGridPts = 750

γ      = 1.0 # RBF Kernel parameter
paramC = 1.0 # Regularization parameter

In [ ]:
# Load Data

# Based on SciKit Learn's Tutorial: Plot classification boundaries with different SVM Kernels (https://scikit-learn.org/stable/auto_examples/svm/plot_svm_kernels.html)
mX = np.array(
    [
        [ 0.4, -0.7],
        [-1.5, -1.0],
        [-1.4, -0.9],
        [-1.3, -1.2],
        [-1.1, -0.2],
        [-1.2, -0.4],
        [-0.5,  1.2],
        [-1.5,  2.1],
        [ 1.0,  1.0],
        [ 1.3,  0.8],
        [ 1.2,  0.5],
        [ 0.2, -2.0],
        [ 0.5, -2.4],
        [ 0.2, -2.3],
        [ 0.0, -2.7],
        [ 1.3,  2.1],
    ]
)

vY = np.array([-1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# Plot Data

hA = Plot2DClsData(mX, vY, figSize = (7.0, 7.0))
hA.set_title('2D Classification Data')
hA.set(xlim = tuGridX, ylim = tuGridY)
hA.set_aspect('equal', adjustable = 'box')
hA.legend();

### Linear Kernel

In [ ]:
# The Kernel Matrix
mK = np.dot(mX, mX.T)
mK = 0.5 * (mK + mK.T)  # Ensure the kernel matrix is symmetric

In [ ]:
# Reference Solution

oSVC = SVC(kernel = 'precomputed', C = paramC, random_state = seedNum)
oSVC = oSVC.fit(mK, vY)

In [ ]:
# Classifier Grid
vXX = np.linspace(tuGridX[0], tuGridX[1], numGridPts)
vYY = np.linspace(tuGridY[0], tuGridY[1], numGridPts)
mXX = np.c_[np.tile(vXX, numGridPts), np.repeat(vYY, numGridPts)]
mZ  = mXX @ mX.T
mZZ = np.reshape(oSVC.predict(mZ), (numGridPts, numGridPts))

In [ ]:
# Plot the Solution

hA = Plot2DClsData(mX, vY, figSize = (7.0, 7.0))
hA.set_title('Reference Classifier')
hA.set(xlim = tuGridX, ylim = tuGridY)
hA.set_aspect('equal', adjustable = 'box')
hA.contourf(vXX, vYY, mZZ, alpha = 0.35, cmap = 'viridis', vmin = -0.05, vmax = 0.05, levels = [-1, 0.0, 1.0], antialiased = True)
hA.contour(vXX, vYY, mZZ, levels = [-1, 0.0, 1.0], colors = 'k', antialiased = True, linewidths = 2.5);

In [ ]:
# Manual Implementation
# vβ, paramB = KernelSVMCVXPY(mK, vY.astype(np.float64), paramC = paramC)
vβ, paramB = KernelSVMCVXPY(mK, vY, paramC = paramC)
vW = mX.T @ vβ

In [ ]:
# Line Parameters
lineSlope     = -(vW[0] / vW[1]).item()
lineIntercept = -(paramB / vW[1]).item()

In [ ]:
# Line in the Box
minY = (tuGridX[0] * lineSlope) + lineIntercept
maxY = (tuGridX[1] * lineSlope) + lineIntercept

In [ ]:
hA = Plot2DClsData(mX, vY, figSize = (7.0, 7.0))
hA.set_title('Reference Classifier')
hA.set(xlim = tuGridX, ylim = tuGridY)
hA.set_aspect('equal', adjustable = 'box')
hA.contourf(vXX, vYY, mZZ, alpha = 0.35, cmap = 'viridis', vmin = -0.05, vmax = 0.05, levels = [-1, 0.0, 1.0], antialiased = True)
hA.contour(vXX, vYY, mZZ, levels = [-1, 0.0, 1.0], colors = 'k', antialiased = True, linewidths = 2.5);
hA.axline((tuGridX[0], minY), (tuGridX[1], maxY), color = 'red', linestyle = '--', linewidth = 2.0, label = 'SVM Classifier (Manual)')
hA.legend();

### RBF Kernel

In [ ]:
### Generate the Kernel Matrix

# Kernel Matrix of RBF Kernel
mK = rbf_kernel(mX, gamma = γ)

In [ ]:
# Reference Solution

oSVC = SVC(kernel = 'precomputed', C = paramC, random_state = seedNum)
oSVC = oSVC.fit(mK, vY)

In [ ]:
# Manual Implementation
vβ, paramB = KernelSVMCVXPY(mK, vY, paramC = paramC)

In [ ]:
# Classifier Grid
vXX = np.linspace(tuGridX[0], tuGridX[1], numGridPts)
vYY = np.linspace(tuGridY[0], tuGridY[1], numGridPts)
mXX = np.c_[np.tile(vXX, numGridPts), np.repeat(vYY, numGridPts)]
mZ  = rbf_kernel(mXX, mX, gamma = γ)
mZZ = np.reshape(oSVC.predict(mZ), (numGridPts, numGridPts))

In [ ]:
# Plot the Solution

hA = Plot2DClsData(mX, vY, figSize = (7.0, 7.0))
hA.set_title('Reference Classifier')
hA.set(xlim = tuGridX, ylim = tuGridY)
hA.set_aspect('equal', adjustable = 'box')
hA.contourf(vXX, vYY, mZZ, alpha = 0.35, cmap = 'viridis', vmin = -0.05, vmax = 0.05, levels = [-1, 0.0, 1.0], antialiased = True)
hA.contour(vXX, vYY, mZZ, levels = [-1, 0.0, 1.0], colors = 'k', antialiased = True, linewidths = 2.5);

In [ ]:
# Prediction of the Manual Implementation
mZZ = np.reshape(np.sign(mZ @ vβ + paramB), (numGridPts, numGridPts))

In [ ]:
# Plot the Solution

hA = Plot2DClsData(mX, vY, figSize = (7.0, 7.0))
hA.set_title('Manual Implementation Classifier')
hA.set(xlim = tuGridX, ylim = tuGridY)
hA.set_aspect('equal', adjustable = 'box')
hA.contourf(vXX, vYY, mZZ, alpha = 0.35, cmap = 'viridis', vmin = -0.05, vmax = 0.05, levels = [-1, 0.0, 1.0], antialiased = True)
hA.contour(vXX, vYY, mZZ, levels = [-1, 0.0, 1.0], colors = 'k', antialiased = True, linewidths = 2.5);